In [ ]:
DAY = 113
MONTH = 3410
YEAR = 41500
D = 11 + 2 # network radius + 4 equal weighted nodes per bin
p = 1 / (2 << (D-1)) # probability of winning on a given epoch

NNODES = 16

REWARD_BZZ = 40
BZZUSD = 0.28

REWARD = REWARD_BZZ * BZZUSD

In [ ]:
# AWS prices for 16 nodes estimate
# https://aws.amazon.com/ec2/pricing/on-demand/

EC2_HOUR = NNODES * 0.0672 / 16 # t4g.large - assume this can run 16 nodes
#M6I_4XLARGE_HOUR = 0.768
EC2_EPOCH = EC2_HOUR * 5 * 152 / 3600
EC2_MONTH = EC2_HOUR * 24 * 30
print(f"Compute cost per month: {EC2_MONTH:0.2f}")

# https://aws.amazon.com/ebs/pricing/

EBS_GB_MONTH = 0.08
EBS_MONTH = EBS_GB_MONTH * 16 * NNODES
EBS_EPOCH = EBS_MONTH * 5 * 152 / (30*86400)
print(f"Storage cost per month: {EBS_MONTH:0.2f}")

COST_EPOCH = EC2_EPOCH + EBS_EPOCH
COST_MONTH = EC2_MONTH + EBS_MONTH
print(f"Total cost per month:   {EC2_MONTH+EBS_MONTH:0.2f}")


INIT = COST_MONTH * 3 # enough cash reserves for 3 months of costs

In [ ]:
print(f"Expected monthly revenue: {MONTH * REWARD * p * NNODES:.02f}")

In [ ]:
from scipy.stats import binom
import numpy as np
from matplotlib import pyplot as plt

## Value at risk, dry spells

In [ ]:
print(f"Prob(no rewards in a month |  1 node): {binom.pmf(0, MONTH, p):.04f}")
print(f"Prob(no rewards in a year  |  1 node): {binom.pmf(0, YEAR, p):.04f}")
print("")
print(f"Prob(no rewards in a month | 16 nodes): {binom.pmf(0, 2*MONTH, 16*p):.04f}")
print(f"Prob(no rewards in a year  | 16 nodes): {binom.pmf(0, YEAR, 16*p):.04f}")

In [ ]:
for d in range(0,10):
    n = (2<<d) // 2
    nwins = int(binom.ppf(0.05, MONTH, n*p))
    print(f"Monthly revenue @5% with {n} nodes: {nwins * REWARD:0.2f}")
    
print("")

for d in range(0,10):
    n = (2<<d) // 2
    nwins = int(binom.ppf(0.01, MONTH, n*p))
    print(f"Monthly revenue @1% with {n} nodes: {nwins*REWARD:0.2f}")

In [ ]:
for d in range(0,10):
    n = (2<<d) // 2
    nwins = int(binom.ppf(0.05, MONTH, n*p))
    cost = n * COST_MONTH / NNODES
    print(f"Monthly VaR@5% with {n} nodes: {INIT + nwins * REWARD - cost:0.2f}")
    
print("")

for d in range(0,10):
    n = (2<<d) // 2
    nwins = int(binom.ppf(0.01, MONTH, n*p))
    cost = n * COST_MONTH / NNODES
    print(f"Monthly VaR@1% with {n} nodes: {INIT + nwins*REWARD - cost:0.2f}")

In [ ]:
for d in range(0,10):
    n = (2<<d) // 2
    nwins = int(binom.ppf(0.05, YEAR, n*p))
    cost = n * COST_MONTH / NNODES
    print(f"Annual VaR@5% with {n} nodes: {INIT + nwins * REWARD - cost*12:0.2f}")
    
print("")

for d in range(0,10):
    n = (2<<d) // 2
    nwins = int(binom.ppf(0.01, YEAR, n*p))
    cost = n * COST_MONTH / NNODES
    print(f"Annual VaR@1% with {n} nodes: {INIT + nwins*REWARD - cost*12:0.2f}")

## Hitting time distribution

In [ ]:
# %%timeit
# 5.28 ms ± 619 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)

ticks = np.arange(1,10*YEAR)
hit_prob = np.zeros(10*YEAR-1)

breakeven = (ticks-(INIT//COST_EPOCH)) / (REWARD//COST_EPOCH) # number of wins needed to exactly break even at time t
breakeven_prob = binom.pmf(breakeven, ticks, p*NNODES) # probability to exactly break even at epoch t

hit_prob = (INIT//COST_EPOCH) * breakeven_prob / ticks # use hitting time theorem

# In practice, the following seems to approximate P(ruin in finite time) to 4 d.p.
# i.e. increasing the time span doesn't change the result
print(f"P(ruin in 10 years) = {hit_prob.sum():.04f}")

In [ ]:
plt.plot(ticks / 113, breakeven_prob)
ax = plt.gca()
ax.set_xlabel("Days")
ax.set_ylabel("P(ruin)")
ax.set_title(f"P(ruin) for {NNODES} nodes with ${INIT:.02f} of operating reserve")
plt.show()